In [1]:
%cd ../src

/Users/shanekercheval/repos/sik-llms/src


/Users/shanekercheval/repos/sik-llms/.venv/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Chat

In [ ]:
from sik_llms import create_client, user_message, ChatChunkResponse

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)

message = user_message("What is the capital of France?")
message

{'role': 'user', 'content': 'What is the capital of France?'}

### Run Synchronously

In [ ]:
client(messages=[message])

ChatResponseSummary(content='The capital of France is Paris.', total_input_tokens=14, total_output_tokens=7, total_input_cost=2.1e-06, total_output_cost=4.2e-06, duration_seconds=0.5634920597076416)

### Run Asynchronously

In [ ]:
responses = []
async for response in client.run_async(messages=[message]):
    if isinstance(response, ChatChunkResponse):
        print(response.content, end="")
        responses.append(response)

The capital of France is Paris.

In [5]:
print(response)

content='The capital of France is Paris.' total_input_tokens=14 total_output_tokens=7 total_input_cost=2.1e-06 total_output_cost=4.2e-06 duration_seconds=0.4606339931488037


### Claude 

In [8]:
client = create_client(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
response = client(messages=[user_message("What is the capital of France?")])
response

ChatResponseSummary(content="The capital of France is Paris. It's not only the capital city but also the largest city in France, known for its iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.", total_input_tokens=14, total_output_tokens=49, total_input_cost=4.2000000000000004e-05, total_output_cost=0.000735, duration_seconds=1.3953912258148193)

# OpenAI Functions

In [8]:
from sik_llms import Function, Parameter, RegisteredClients

weather_tool = Function(
    name="get_weather",
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name="location",
            type="string",
            required=True,
            description="The city and country for weather info.",
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.OPENAI_FUNCTIONS,
    model_name='gpt-4o-mini',
    functions=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
response

FunctionCallResponse(input_tokens=60, output_tokens=17, input_cost=9e-06, output_cost=1.0199999999999999e-05, duration_seconds=0.8786740303039551, function_call=FunctionCallResult(name='get_weather', arguments={'location': 'Paris, France'}, call_id='call_OakECL19U7xSzKN4gy4NXtZc'), message=None)

---

# Claude Functions

In [7]:
from sik_llms import (
    create_client,
    user_message,
    Function,
    Parameter,
    RegisteredClients,
)

weather_tool = Function(
    name="get_weather",
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name="location",
            type="string",
            required=True,
            description="The city and country for weather info.",
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.ANTHROPIC_FUNCTIONS,
    model_name='claude-3-7-sonnet-latest',
    functions=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
response

FunctionCallResponse(input_tokens=394, output_tokens=40, input_cost=0.001182, output_cost=0.0006000000000000001, duration_seconds=1.7429931163787842, function_call=FunctionCallResult(name='get_weather', arguments={'location': 'Paris, France'}, call_id='toolu_011pXGg6U2QsLt2DCVtS36AE'), message=None)

---

# Structured Outputs via OpenAI

In [9]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# Create an instance of the wrapper
client = create_client(
    model_name='gpt-4o-mini',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = client(messages=messages)
response

ChatResponseSummary(input_tokens=92, output_tokens=18, input_cost=1.38e-05, output_cost=1.08e-05, duration_seconds=1.2957699298858643, content=StructuredOutputResponse(parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']), refusal=None))

---

# Reasoning via OpenAI

In [5]:
from sik_llms import (
    create_client,
    user_message,
    ChatChunkResponse,
    ChatResponseSummary,
    ReasoningEffort,
)

client = create_client(
    model_name='o3-mini',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None
async for response in client.run_async(messages=messages):
    if isinstance(response, ChatChunkResponse):
        print(response.content, end="")
    elif isinstance(response, ChatResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")

Let's break it down step by step:

1. First, calculate the multiplications:
   - 3 * 4 = 12
   - 5 * 6 = 30

2. Now substitute these back into the expression:
   1 + 2 + 12 + 30

3. Finally, add them together:
   - 1 + 2 = 3
   - 3 + 12 = 15
   - 15 + 30 = 45

So, the answer is 45.

In [6]:
summary

ChatResponseSummary(content="Let's break it down step by step:\n\n1. First, calculate the multiplications:\n   - 3 * 4 = 12\n   - 5 * 6 = 30\n\n2. Now substitute these back into the expression:\n   1 + 2 + 12 + 30\n\n3. Finally, add them together:\n   - 1 + 2 = 3\n   - 3 + 12 = 15\n   - 15 + 30 = 45\n\nSo, the answer is 45.", total_input_tokens=28, total_output_tokens=111, total_input_cost=3.08e-05, total_output_cost=0.0004884, duration_seconds=1.6389861106872559)

---

# Reasoning via Claude

In [2]:
from sik_llms import (
    create_client,
    user_message,
    ChatChunkResponse,
    ChatResponseSummary,
    ReasoningEffort,
)

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None
async for response in client.run_async(messages=messages):
    if isinstance(response, ChatChunkResponse):
        print(response.content, end="")
    elif isinstance(response, ChatResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")

I'll evaluate this expression step by step.

1 + 2 + (3 * 4) + (5 * 6)

First, I'll compute the parts in parentheses.
(3 * 4) = 12
(5 * 6) = 30

So the expression becomes:
1 + 2 + 12 + 30

Now I'll add these numbers:
1 + 2 = 3
3 + 12 = 15
15 + 30 = 45

So the final result is 45.To solve this expression, I'll follow the order of operations:

1 + 2 + (3 * 4) + (5 * 6)

First, I'll evaluate the expressions in parentheses:
(3 * 4) = 12
(5 * 6) = 30

Now I can add all terms:
1 + 2 + 12 + 30 = 45

The answer is 45.

---